In [1]:
%use adventOfCode

In [2]:
val puzzle = AocClient.fromEnv().interactiveDay(2024, 23)


In [3]:
val testInput = """kh-tc
qp-kh
de-cg
ka-co
yn-aq
qp-ub
cg-tb
vc-aq
tb-ka
wh-tc
yn-cg
kh-ub
ta-co
de-co
tc-td
tb-wq
wh-td
ta-ka
td-qp
aq-cg
wq-ub
ub-vc
de-ta
wq-aq
wq-vc
wh-yn
ka-de
kh-ta
co-tc
wh-qp
tb-vc
td-yn"""

fun parse(input: String) = input.lines().map { it.split('-') }.flatMap { (a, b) -> listOf(a to b, b to a) }
    .groupBy({ it.first }, { it.second })
parse(testInput)

{kh=[tc, qp, ub, ta], tc=[kh, wh, td, co], qp=[kh, ub, td, wh], de=[cg, co, ta, ka], cg=[de, tb, yn, aq], ka=[co, tb, ta, de], co=[ka, ta, de, tc], yn=[aq, cg, wh, td], aq=[yn, vc, cg, wq], ub=[qp, kh, wq, vc], tb=[cg, ka, wq, vc], vc=[aq, ub, wq, tb], wh=[tc, td, yn, qp], ta=[co, ka, de, kh], td=[tc, wh, qp, yn], wq=[tb, ub, aq, vc]}

In [4]:
fun findTriplets(parsed: Map<String, List<String>>): List<List<String>> {
    fun connections(node: String) = parsed.getOrDefault(node, emptyList())
    return parsed.flatMap { (node1, connections) ->
        connections.filter { node2 -> node1 < node2 }.flatMap { node2 ->
            connections(node2).filter { node3 -> node2 < node3 }.filter { node3 -> node1 in connections(node3) }
                .map { node3 -> listOf(node1, node2, node3) }
        }
    }
}

fun part1(input: String): Int {
    val parsed = parse(input)
    val triplets = findTriplets(parsed)
    return triplets.count { triplet -> triplet.any { it[0] == 't' } }
}

part1(testInput)

7

In [5]:
part1(puzzle.input())

893

In [6]:
class GroupFinder(val graph: Map<String, List<String>>) {
    fun connections(node: String) = graph.getOrDefault(node, emptyList())

    fun findGroups(size: Int): List<List<String>> =
        graph.flatMap { (node, _) -> expandGroup(node, listOf(node), size - 1) }

    fun expandGroup(prevNode: String, selected: List<String>, remainingSize: Int): List<List<String>> =
        if (remainingSize == 0)
            listOf(selected)
        else
            connections(prevNode).filter { newNode -> prevNode < newNode }
                .filter { newNode -> connections(newNode).containsAll(selected) }
                .flatMap { newNode -> expandGroup(newNode, selected + newNode, remainingSize - 1) }
}

fun part2(input: String): List<String> {
    val parsed = parse(input)
    val finder = GroupFinder(parsed)
    return (3..parsed.size).asSequence().map {
        finder.findGroups(it)
    }.first { it.size == 1 }.single()
}


In [7]:
part2(testInput)

[co, de, ka, ta]

In [8]:
part2(puzzle.input()).joinToString(",")

cw,dy,ef,iw,ji,jv,ka,ob,qv,ry,ua,wt,xz